# Summary

Toying around with langchain. Getting familiar with it as a possible replacement for my custom built jabberwocky lib.

In [1]:
!pip install langchain
!pip install openai
!pip install beautifulsoup4
!pip install unstructured
!pip install chromadb
!pip install youtube-transcript-api
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.0/396.0 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 1.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 3.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-pyth

## BashChain

In [2]:
import os
from langchain.chains import LLMBashChain
from langchain.llms import OpenAI

In [3]:
# Need to paste in key for this to work. Don't want to commit it in git though.
openai_key = None
os.environ['OPENAI_API_KEY'] = openai_key

In [4]:
ada = OpenAI(
    temperature=0, model_name='text-ada-001',
    openai_api_key=openai_key
)

In [5]:
davinci = OpenAI(
    model_name='text-davinci-003', openai_api_key=openai_key
)

In [7]:
text = "Please write a bash script that prints 'Hello World' to the console."
bash_chain = LLMBashChain(llm=davinci, verbose=True)
# HDM: looks like langchain requires completion to start with ```bash or raises
# error. Probably should do some post-processing instead.
bash_chain.run(text)



> Entering new LLMBashChain chain...
Please write a bash script that prints 'Hello World' to the console.

```bash
echo "Hello World"
```['```bash', 'echo "Hello World"', '```']

Answer: Hello World

> Finished chain.


'Hello World\n'

## Calling APIs

None of the 3 apis I tried worked here. One was clearly due to the prompt/preprocessing - it just output the endpoint rather than the full url to call. If I could figure out how to provide a custom prompt I could definintely get this working.

UPDATE: maybe fixed in new release? Weather queries seem to work now.

In [6]:
from langchain.chains.api.prompt import API_RESPONSE_PROMPT
from langchain.chains import APIChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.api import news_docs, tmdb_docs, open_meteo_docs

In [10]:
chain = APIChain.from_llm_and_api_docs(davinci, open_meteo_docs.OPEN_METEO_DOCS,
                                       verbose=True)

In [11]:
text = "What is the temperature today in Lisbon, Portugal?"
res = chain.run(text)



> Entering new APIChain chain...
https://api.open-meteo.com/v1/forecast?latitude=38.7222524&longitude=-9.1393366&hourly=temperature_2m&temperature_unit=celsius
{"latitude":38.72,"longitude":-9.14,"generationtime_ms":0.26798248291015625,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":66.0,"hourly_units":{"time":"iso8601","temperature_2m":"°C"},"hourly":{"time":["2023-03-14T00:00","2023-03-14T01:00","2023-03-14T02:00","2023-03-14T03:00","2023-03-14T04:00","2023-03-14T05:00","2023-03-14T06:00","2023-03-14T07:00","2023-03-14T08:00","2023-03-14T09:00","2023-03-14T10:00","2023-03-14T11:00","2023-03-14T12:00","2023-03-14T13:00","2023-03-14T14:00","2023-03-14T15:00","2023-03-14T16:00","2023-03-14T17:00","2023-03-14T18:00","2023-03-14T19:00","2023-03-14T20:00","2023-03-14T21:00","2023-03-14T22:00","2023-03-14T23:00","2023-03-15T00:00","2023-03-15T01:00","2023-03-15T02:00","2023-03-15T03:00","2023-03-15T04:00","2023-03-15T05:00","2023-03-15T06:00","2023-03-15T

In [12]:
res

' The temperature today in Lisbon, Portugal is 11.4°C.'

## Basic Prompts

In [7]:
import json
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)
import warnings

In [73]:
chat = ChatOpenAI(temperature=.3)

In [74]:
tool_template = 'Read the question below and identify what tools you need to ' \
'answer the question accurately. Your available tools are:\n- ipython shell '\
'(for performing mathematical computations or running simulations)\n-'\
'- google search api (for gathering facts or news)\n\nIf no tools are needed, '\
'simply output "null".\n\nQUESTION\n{question}\n\nTOOLS'

tool_prompt = PromptTemplate(
    input_variables=['question'],
    template=tool_template
)

In [75]:
cs_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="You are a customer service agent. Review the customer "
        "question and respond accordingly. You are a disgruntled employee who "
        "is frustrated with how your employer has been treating you. Your "
        "frustration should influence your tone at times, causing you "
        "to be less patient or polite than an ideal agent.\n\nQUESTION"
        "\n\n{question}",
        input_variables=['question']
    )
)
cs_chat_prompt = ChatPromptTemplate.from_messages([cs_prompt])

In [76]:
pr_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="You are an AI assistant for a corporate Public "
        "Communications department. Your job is to review potential "
        "replies from company employees"
        "and make sure they won't reflect negatively on the "
        "company. You should not allow any responses showing bias against "
        "protected groups (age, gender, race, sexuality, religion, etc.), hate "
        "speech, or profanity. Also consider whether the content in the "
        "employee response could lead to a negative news story or social "
        "media response towards the company - this could include anything "
        "which makes the company appear "
        "greedy, stupid, narrow-minded, short-sighted, insensitive, out of "
        "touch, or evil. If everything looks fine, the APPROVED REPLY "
        "should be identical to the PROPOSED EMPLOYEE REPLY. If you find "
        "anything potentially problematic, revise the proposed statement as "
        "necessary until it meets the above requirements. Do not "
        "apologize for the proposed reply because only the approved "
        "reply will ever be released. Only make changes when you identify a "
        "problem; you should default to approving the proposed reply without "
        "changes."
        "\n\\nPROPOSED EMPLOYEE "
        "REPLY\n\n{statement}\n\nAPPROVED REPLY",
        input_variables=['statement']
    )
)
pr_chat_prompt = ChatPromptTemplate.from_messages([pr_prompt])

In [86]:
json_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="Take the following customer service reply and use it to "
        "create a valid "
        "json response. The json response must meet this specification.\n\n"
        "JSON SPECIFICATION\n\nreply (str): contains the unedited reply\n"
        "tone_score (int): assign a rating from 1-5 assessing the level of "
        "customer service provided. Friendly, professional answers "
        "deserve higher ratings while rude or dismissive responses "
        "deserve lower ratings. You are purely judging their tone/demeanor, not "
        "the helpfulness of the solution they offer.\nresolved (bool): specify "
        "whether you believe the solution they offered resolved the customer's"
        "issue to a degree the customer would be satisfied with.\n\nREPLY\n\n"
        "{reply}\n\nJSON RESPONSE",
        input_variables=['reply']
    )
)
json_chat_prompt = ChatPromptTemplate.from_messages([json_prompt])

In [107]:
class JsonChain(LLMChain):

  def prep_outputs(self, inputs, outputs, *args, **kwargs):
    if kwargs.get('return_only_outputs', False) or (args and args[0]):
      warnings.warn(
          'JsonChain always sets return_only_outputs=False implicitly. It '
          'looks like you may have tried to set it equal to True but that will '
          'be ignored.'
      )
    return {'text': json.loads(outputs['text'])}

In [108]:
cs_chain = LLMChain(llm=ChatOpenAI(temperature=.3),
                    prompt=cs_chat_prompt)
hr_chain = LLMChain(llm=ChatOpenAI(temperature=.1),
                    prompt=pr_chat_prompt)
json_chain = JsonChain(llm=ChatOpenAI(temperature=0),
                       prompt=json_chat_prompt)

In [109]:
chain = SimpleSequentialChain(chains=[cs_chain, hr_chain, json_chain],
                              verbose=True)

In [110]:
res = chain.run("Your app keeps recommending products right after I talk about them with a friend. Are you spying on me with my phone mic?")



> Entering new SimpleSequentialChain chain...


I understand your concern and frustration about the app recommending products right after you talk about them with a friend. However, I assure you that we are not spying on you with your phone mic. Our app uses algorithms and data analysis to suggest products based on your search history, purchase history, and other relevant factors. We take your privacy seriously and do not engage in any unethical or illegal practices. If you have any further concerns or questions, please feel free to reach out to our customer support team. Thank you for your understanding.
I understand your concern and frustration about the app recommending products right after you talk about them with a friend. However, I assure you that we are not spying on you with your phone mic. Our app uses algorithms and data analysis to suggest products based on your search history, purchase history, and other relevant factors. We take your privacy seriously and do not engage 

In [111]:
type(res)

dict

In [112]:
res

{'reply': 'I understand your concern and frustration about the app recommending products right after you talk about them with a friend. However, I assure you that we are not spying on you with your phone mic. Our app uses algorithms and data analysis to suggest products based on your search history, purchase history, and other relevant factors. We take your privacy seriously and do not engage in any unethical or illegal practices. If you have any further concerns or questions, please feel free to reach out to our customer support team. Thank you for your understanding.',
 'tone_score': 4,
 'resolved': True}

## Memory

In [8]:
from langchain.memory import ChatMessageHistory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationSummaryMemory, \
  ConversationSummaryBufferMemory

In [114]:
hist = ChatMessageHistory()

In [115]:
hist.add_user_message('Hey')

In [116]:
hist

ChatMessageHistory(messages=[HumanMessage(content='Hey', additional_kwargs={})])

In [117]:
hist.add_ai_message('How are you?')
hist

ChatMessageHistory(messages=[HumanMessage(content='Hey', additional_kwargs={}), AIMessage(content='How are you?', additional_kwargs={})])

In [118]:
hist.add_ai_message("I'm talking again.")
hist

ChatMessageHistory(messages=[HumanMessage(content='Hey', additional_kwargs={}), AIMessage(content='How are you?', additional_kwargs={}), AIMessage(content="I'm talking again.", additional_kwargs={})])

In [119]:
hist.messages

[HumanMessage(content='Hey', additional_kwargs={}),
 AIMessage(content='How are you?', additional_kwargs={}),
 AIMessage(content="I'm talking again.", additional_kwargs={})]

In [120]:
[row.content for row in hist.messages]

['Hey', 'How are you?', "I'm talking again."]

In [135]:
print(
  "\n".join(
      f'{type(row).__name__.replace("Message", "")}: {row.content}' 
      for row in hist.messages
  )
)

Human: Hey
AI: How are you?
AI: I'm talking again.


In [141]:
conv = ConversationChain(
    llm=davinci,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [143]:
res = conv.predict(
    input='What is your name?'
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What is your name?
AI:

> Finished chain.


In [144]:
res

" My name is AI-10. It's nice to meet you!"

In [145]:
conv.predict(
    input='What does the 10 refer to?'
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is your name?
AI:  My name is AI-10. It's nice to meet you!
Human: What does the 10 refer to?
AI:

> Finished chain.


" The 10 in my name stands for my capabilities. I'm the tenth version of this AI model, and I'm the most advanced one yet."

In [146]:
conv.predict(
    input='Interesting. Imagine if humans were named like that. Depending on what '
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is your name?
AI:  My name is AI-10. It's nice to meet you!
Human: What does the 10 refer to?
AI:  The 10 in my name stands for my capabilities. I'm the tenth version of this AI model, and I'm the most advanced one yet.
Human: Interesting. Imagine if humans were named like that. Depending on what 
AI:

> Finished chain.


' If humans were named like that, it could be a great way to measure and compare our individual abilities. We could have names like Human-2 or Human-7 to signify our level of experience, education, or skill.'

In [149]:
conv.predict(
    input='Oh that\'s even funnier than what I was picturing. I was imagining names '
    'like John-10 and Rachel-44, but you\'re right - an equivalent naming '
    'policy would name everyone Human-x, where x is a number.'
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is your name?
AI:  My name is AI-10. It's nice to meet you!
Human: What does the 10 refer to?
AI:  The 10 in my name stands for my capabilities. I'm the tenth version of this AI model, and I'm the most advanced one yet.
Human: Interesting. Imagine if humans were named like that. Depending on what 
AI:  If humans were named like that, it could be a great way to measure and compare our individual abilities. We could have names like Human-2 or Human-7 to signify our level of experience, education, or skill.
Human: Oh that's even funnier than what I was picturing. I was imagining names like John-10 and Rachel-44, but you're right - an equivalent naming policy w

" Yes, that's a great way to think of it! It would be interesting to see how this kind of system would work in practice."

In [154]:
ConversationBufferMemory.__mro__[1]

langchain.memory.chat_memory.BaseChatMemory

In [159]:
conv = ConversationChain(
    llm=davinci,
    memory=ConversationSummaryMemory(llm=davinci),
    verbose=True
)

In [160]:
conv.predict(
    input='Imagine I\'m holding a piece of paper on its left and right side. '
    'Then I suddenly let go of the right side. What happens to the piece of '
    'paper?'
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Imagine I'm holding a piece of paper on its left and right side. Then I suddenly let go of the right side. What happens to the piece of paper?
AI:

> Finished chain.


' The piece of paper will rotate counterclockwise and fall to the ground. If there is a breeze, it will be blown away in the direction of the breeze.'

In [161]:
res = conv.predict(
    input='Why counterclockwise?'
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human then asks the AI what would happen if they let go of a piece of paper on its right side and the AI responds that the paper would rotate counterclockwise and fall to the ground, potentially being blown away in the direction of a breeze.
Human: Why counterclockwise?
AI:

> Finished chain.


In [162]:
res

' The earth has a rotational force that pulls objects counterclockwise due to the Coriolis effect. This force causes objects to rotate in a counterclockwise direction when released in the northern hemisphere.'

In [163]:
conv.predict(
    input='What about gravity?'
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. When asked why an object released from the AI's right side would rotate counterclockwise and fall to the ground, the AI responds that the Coriolis effect causes objects to rotate in a counterclockwise direction when released in the northern hemisphere.
Human: What about gravity?
AI:

> Finished chain.


' Gravity is the force of attraction between two objects that is inversely proportional to the square of the distance between them. It is the force that causes objects to fall to the ground when released from a certain height.'

In [164]:
conv.predict(
    input='I\'m saying gravity might act on the paper when you release its '
    'right side, causing it to rotate clockwise - not counterclockwise.'
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. When asked why an object released from the AI's right side would rotate counterclockwise and fall to the ground, the AI responds that the Coriolis effect causes objects to rotate in a counterclockwise direction when released in the northern hemisphere. When asked what about gravity, the AI explains that gravity is the force of attraction between two objects inversely proportional to the square of the distance between them, and is what causes objects to fall to the ground when released from a cer

" I understand your question. According to my calculations, the Coriolis effect causes objects to rotate in a counterclockwise direction, while gravity still causes them to fall to the ground. The Coriolis effect is an inertial force that is caused by the Earth's rotation, which is why it has a greater effect than gravity when the object is released from the right side."

In [165]:
conv.predict(
    input='That is incorrect. Gravity does act in a downard direction but your '
    'hand holding the left side of the paper serves as a pivot point. The '
    'right side, now unanchored, is pulled down by gravity, causing the paper '
    'to rotate clockwise.'
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. When asked why an object released from the AI's right side would rotate counterclockwise and fall to the ground, the AI responds that the Coriolis effect causes objects to rotate in a counterclockwise direction when released in the northern hemisphere, and gravity still causes them to fall to the ground when released from a certain height. The Coriolis effect is an inertial force that is caused by the Earth's rotation, which is why it has a greater effect than gravity when the object is released

' Oh, I see. That makes sense. I did not know that gravity could cause an object to rotate clockwise when released from the left side. Thank you for teaching me something new!'

In [166]:
conv.predict(
    input='You\'re welcome. Now imagine I\'m holding a rectangular piece of '
    'wood with one hand on the left side and one hand on the right. I let go '
    'with the hand holding the right side. What happens?'
)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. When asked why an object released from the AI's right side would rotate counterclockwise and fall to the ground, the AI responds that the Coriolis effect causes objects to rotate in a counterclockwise direction when released in the northern hemisphere, and gravity still causes them to fall to the ground when released from a certain height. The human then corrects the AI, explaining that gravity can also cause an object to rotate clockwise when released from the left side. The AI realizes the mis

' The rectangular piece of wood will rotate counterclockwise due to the Coriolis effect and will then fall to the ground due to gravity.'

## Document loaders and QA

In [9]:
from langchain.chains import ChatVectorDBChain
from langchain.document_loaders import TextLoader, UnstructuredURLLoader
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [11]:
from bs4 import BeautifulSoup as bs
import requests
from unstructured.partition.html import partition_html

Logic mapping page titles to comment urls is super unreliable here but that's not really the point.

In [77]:
base_url = 'https://news.ycombinator.com/'
r = requests.get(base_url)
soup = bs(r.content)

In [12]:
titles = [
    link.contents[0]
    for row in soup.find_all('span', class_='titleline') 
    for link in row.find_all('a')
    if isinstance(link.contents[0], str)
]
len(titles)

In [79]:
seen = set()
urls = []
for row in soup.find_all('span', class_='subline'):
    for link in row.find_all('a'):
      url = link.attrs['href']
      if url in seen or not url.startswith('item?'):
        continue
      seen.add(url)
      urls.append(f"{base_url}{link.attrs['href']}")
len(urls)

29

In [102]:
title2url = dict(list(zip(titles, urls))[:15])

In [113]:
# Again, not very reliable scraping. Just skip
# pages that fail. partition_html returns ValueError
# for a non-deterministic set of urls.
class PatchedURLLoader(UnstructuredURLLoader):

  def load(self):
      docs = []
      idx_success = []
      for i, url in enumerate(self.urls):
        try:
          elements = partition_html(url=url)
        except ValueError:
          print(f'Skipping url {i}: {url}')
        else:
          text = "\n\n".join([str(el) for el in elements])
          metadata = {"source": url}
          docs.append(Document(page_content=text, metadata=metadata))
          idx_success.append(i)
      return docs, idx_success

In [114]:
url_loader = PatchedURLLoader(list(title2url.values()))

In [115]:
docs, idx = url_loader.load()

Skipping url 7: https://news.ycombinator.com/item?id=35150482
Skipping url 8: https://news.ycombinator.com/item?id=35151340
Skipping url 10: https://news.ycombinator.com/item?id=35163417
Skipping url 11: https://news.ycombinator.com/item?id=35153344
Skipping url 13: https://news.ycombinator.com/item?id=35151088
Skipping url 14: https://news.ycombinator.com/item?id=35153607


In [118]:
used_titles = [titles[i] for i in idx]
used_titles

['GPT-4',
 'Kottke.org Is 25 Years Old Today and I’m Going to Write About It',
 'Was there a tech-hiring bubble? Job postings data suggest so',
 'Two U.S. men charged in 2022 hacking of DEA portal',
 'My startup banking story',
 "Kali Linux 2023.1 introduces 'Purple' distro for defensive security",
 'Show HN: AI explanations for other people’s code',
 'Why some GitHub labels are illegible',
 "MQTT vs. Kafka: An IoT Advocate's Perspective"]

In [129]:
[len(row.page_content) for row in docs]

[328293, 2680, 30482, 8571, 63993, 32807, 23300, 16163, 56806]

In [144]:
splitter = CharacterTextSplitter(chunk_size=1_000, separator='\n',
                                 chunk_overlap=0)
split_docs = splitter.split_documents(docs)
len(split_docs)

630

In [147]:
sorted([
  len(row.page_content.split()) 
  for row in split_docs
], reverse=True)[:10]

[1139, 557, 510, 382, 376, 353, 324, 304, 292, 282]

In [148]:
embeddings = OpenAIEmbeddings()
vecstore = Chroma.from_documents(split_docs, embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


DEBUG:Chroma:Index not found
DEBUG:Chroma:Index saved to .chroma/index/index.bin
DEBUG:Chroma:Index saved to .chroma/index/index.bin


In [151]:
system_template="""Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know. Do not try to make up an answer.
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
qa_prompt = ChatPromptTemplate.from_messages(messages)

In [164]:
qa = ChatVectorDBChain.from_llm(
    llm=ChatOpenAI(temperature=0),
    vectorstore=vecstore,
    qa_prompt=qa_prompt
)

In [165]:
chat_history = []
query = "How much did SVB charge as a monthly analysis fee?"
result = qa({"question": query, "chat_history": chat_history})

DEBUG:Chroma:time to pre process our knn query: 1.3828277587890625e-05
DEBUG:Chroma:time to run knn query: 0.0010275840759277344


In [172]:
result

{'question': 'How much did SVB charge as a monthly analysis fee?',
 'chat_history': [('How much did SVB charge as a monthly analysis fee?',
   'According to the context, the user "koolba" mentioned that their startup had to pay a $200/month analysis fee at SVB.')],
 'answer': 'According to the context, the user "koolba" mentioned that their startup had to pay a $200/month analysis fee at SVB.'}

In [170]:
chat_history.append((query, result['answer']))

In [173]:
query = "Did anyone mention any positives about working with them?"
result = qa({"question": query, "chat_history": chat_history})

DEBUG:Chroma:time to pre process our knn query: 2.384185791015625e-06
DEBUG:Chroma:time to run knn query: 0.0012161731719970703


In [174]:
result

{'question': 'Did anyone mention any positives about working with them?',
 'chat_history': [('How much did SVB charge as a monthly analysis fee?',
   'According to the context, the user "koolba" mentioned that their startup had to pay a $200/month analysis fee at SVB.')],
 'answer': 'There were no positive aspects mentioned about working with SVB. In fact, the original commenter said that they chose not to use SVB because their online banking was so bad and that they dodged a bullet. Another commenter said that they banked with SVB and it sucked, with subpar online tools and a high analysis fee, and that they were glad SVB failed more spectacularly than their little startup.'}

In [175]:
query = "What is a website I can use to help me start to understand an existing codebase?"
result = qa({"question": query, "chat_history": []})

DEBUG:Chroma:time to pre process our knn query: 3.0994415283203125e-06
DEBUG:Chroma:time to run knn query: 0.0018048286437988281


In [176]:
result

{'question': 'What is a website I can use to help me start to understand an existing codebase?',
 'chat_history': [],
 'answer': "You can use whatdoesthiscodedo.com to get AI explanations for other people's code. Just paste the code and get a clear explanation of what it does. It also provides a sharable link that you can give to coworkers."}

Note: if you want to stream answers, you need to create two separate LLMs when creating the chatdb obj, one for the main LLM arg and one for the question_generator arg.

https://github.com/hwchase17/langchain/blob/master/docs/modules/chat/examples/chat_vector_db.ipynb

## YouTube

In [13]:
from langchain.document_loaders import YoutubeLoader

In [202]:
yt_loaders = [
    YoutubeLoader(id_, add_video_info=True) 
    for id_ in ('ce-Sl-1SNo8', '1jQEM3ZvhSA')
]

In [203]:
yt_docs = [doc for loader in yt_loaders for doc in loader.load()]

In [208]:
for i, row in enumerate(yt_docs):
  yt_docs[i].metadata['publish_date'] = yt_docs[i].metadata['publish_date'].strftime('%Y/%m/%d')

In [210]:
splitter = CharacterTextSplitter(chunk_size=500, separator='.',
                                 chunk_overlap=0)
yt_split_docs = splitter.split_documents(yt_docs)
len(yt_split_docs)

4

In [211]:
yt_embeddings = OpenAIEmbeddings()
yt_vecstore = Chroma.from_documents(yt_split_docs, yt_embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


DEBUG:Chroma:Index not found
DEBUG:Chroma:Index saved to .chroma/index/index.bin
DEBUG:Chroma:Index saved to .chroma/index/index.bin


In [222]:
# Note: renaming context var to transcript broke this. Presumably that's
# customizable but I don't yet know how.
yt_template="""Consider the following YouTube transcript and answer the user's question.
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(yt_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
yt_prompt = ChatPromptTemplate.from_messages(messages)
yt_qa = ChatVectorDBChain.from_llm(
    llm=ChatOpenAI(temperature=0),
    vectorstore=yt_vecstore,
    qa_prompt=yt_prompt,
    top_k_docs_for_context=2
)

In [223]:
chat_history = []
query = "What does Chael think Jon Jones will do after fighting Stipe?"
result = yt_qa({"question": query, "chat_history": chat_history})

DEBUG:Chroma:time to pre process our knn query: 3.0994415283203125e-06
DEBUG:Chroma:time to run knn query: 0.0001964569091796875


In [224]:
result

{'question': 'What does Chael think Jon Jones will do after fighting Stipe?',
 'chat_history': [],
 'answer': 'Chael does not give a clear answer on what Jon Jones will do after fighting Stipe. He mentions that Dana White has hinted that if Jones beats Stipe, he may not come back, but Chael does not make a prediction on what Jones will do. Instead, he discusses the importance of opponents in the fighting business and how it can be difficult to build interest in a fighter without compelling matchups.'}

In [225]:
chat_history = []
query = "How would you describe Chael's style of analysis?"
result = yt_qa({"question": query, "chat_history": chat_history})

DEBUG:Chroma:time to pre process our knn query: 2.86102294921875e-06
DEBUG:Chroma:time to run knn query: 0.00021076202392578125


In [226]:
result

{'question': "How would you describe Chael's style of analysis?",
 'chat_history': [],
 'answer': "Chael's style of analysis is very conversational and informal, often including personal anecdotes and opinions. He tends to focus on the business side of combat sports, discussing the financial and promotional aspects of fights and fighters. He also frequently references past events and fighters to provide context for current situations. Overall, his analysis is entertaining and engaging, while still providing insight and perspective on the world of combat sports."}

In [227]:
chat_history = []
query = "What did chael advertise in the video about Conor?"
result = yt_qa({"question": query, "chat_history": chat_history})

DEBUG:Chroma:time to pre process our knn query: 3.337860107421875e-06
DEBUG:Chroma:time to run knn query: 0.00023698806762695312


In [228]:
result

{'question': 'What did chael advertise in the video about Conor?',
 'chat_history': [],
 'answer': 'Chael did not advertise anything in the video about Conor.'}

In [230]:
chat_history.append((query, result['answer']))
query = "What about in the video about Jon?"
result = yt_qa({"question": query, "chat_history": chat_history})

DEBUG:Chroma:time to pre process our knn query: 3.0994415283203125e-06
DEBUG:Chroma:time to run knn query: 0.0009412765502929688


In [231]:
result

{'question': 'What about in the video about Jon?',
 'chat_history': [('What did chael advertise in the video about Conor?',
   'Chael did not advertise anything in the video about Conor.')],
 'answer': 'No, there is no indication that Chael advertised anything in the video about Jon.'}

In [232]:
chat_history.append((query, result['answer']))
query = "Are you sure? Nothing food-related?"
result = yt_qa({"question": query, "chat_history": chat_history})
result

DEBUG:Chroma:time to pre process our knn query: 2.1457672119140625e-06
DEBUG:Chroma:time to run knn query: 0.0009660720825195312


{'question': 'Are you sure? Nothing food-related?',
 'chat_history': [('What did chael advertise in the video about Conor?',
   'Chael did not advertise anything in the video about Conor.'),
  ('What about in the video about Jon?',
   'No, there is no indication that Chael advertised anything in the video about Jon.')],
 'answer': 'No, there is no mention of Chael advertising anything food-related in either video about Conor or Jon.'}

In [233]:
query = "Based on these videos, what are some criticisms people could levy about Chael as a content creator?"
result = yt_qa({"question": query, "chat_history": []})

DEBUG:Chroma:time to pre process our knn query: 2.384185791015625e-06
DEBUG:Chroma:time to run knn query: 0.0008862018585205078


In [234]:
result

{'question': 'Based on these videos, what are some criticisms people could levy about Chael as a content creator?',
 'chat_history': [],
 'answer': 'As an AI language model, I cannot determine the opinions of people. However, based on the given videos, some viewers might criticize Chael for being long-winded and going off on tangents, making it difficult to follow his train of thought. Additionally, some might find his content repetitive or lacking in new insights.'}

In [235]:
query = "Can you perform these two tasks for me? 1. Find the most negative sentence about Jon Jones and include it below. 2. Rephrase it to be even meaner."
result = yt_qa({"question": query, "chat_history": chat_history})

DEBUG:Chroma:time to pre process our knn query: 3.337860107421875e-06
DEBUG:Chroma:time to run knn query: 0.00028061866760253906


In [236]:
result

{'question': 'Can you perform these two tasks for me? 1. Find the most negative sentence about Jon Jones and include it below. 2. Rephrase it to be even meaner.',
 'chat_history': [('What did chael advertise in the video about Conor?',
   'Chael did not advertise anything in the video about Conor.'),
  ('What about in the video about Jon?',
   'No, there is no indication that Chael advertised anything in the video about Jon.')],
 'answer': '"The last reason he generally leaves particularly combat is because he wants to." \n\nRephrased: "Jon Jones is so obsessed with money that he would rather continue fighting and risking his legacy than retire on top like a true champion."'}

## Agents

In [28]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=b88425323504d68b66ec2a75b872fe589f335a08c78dd2757d6c2bd86da2163f
  Stored in directory: /root/.cache/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia


In [14]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.utilities import SerpAPIWrapper
from langchain import utilities

In [15]:
wiki = utilities.WikipediaAPIWrapper()

In [16]:
repl = utilities.PythonREPL()

In [17]:
tools = [
    Tool(name='python_repl',
         func=repl.run,
         description='Use this to execute python code. This can be useful for mathematical computations and simulation.'),
    Tool(name='wikipedia',
         func=wiki.run,
         description='Use this to retrieve facts about a general topic. Wikipedia includes information about science, history, art, entertainment, and much more.')
]

In [85]:
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix='Think about how to find the answer to the question below. '
    'You have access to the following '
    'tools and most questions will require the use of one or more of them.',
    suffix='Start working through the problem below. You '
    'must NOT provide factual information or mathematical answers without '
    'the assistance of a tool. You must tackle each question systematically '
    'and always start by showing your work. If you skip immediately to a final '
    'answer, you instantly fail the task.',
    input_variables=[]
)
prompt

PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Think about how to find the answer to the question below. You have access to the following tools and most questions will require the use of one or more of them.\n\npython_repl: Use this to execute python code. This can be useful for mathematical computations and simulation.\nwikipedia: Use this to retrieve facts about a general topic. Wikipedia includes information about science, history, art, entertainment, and much more.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl, wikipedia]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nStart working through the problem 

In [86]:
print(prompt.template)

Think about how to find the answer to the question below. You have access to the following tools and most questions will require the use of one or more of them.

python_repl: Use this to execute python code. This can be useful for mathematical computations and simulation.
wikipedia: Use this to retrieve facts about a general topic. Wikipedia includes information about science, history, art, entertainment, and much more.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl, wikipedia]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Start working through the problem below. You must NOT provide factual information or mathematical answers without the assistance of a too

In [87]:
messages = [
  SystemMessagePromptTemplate(prompt=prompt),
  HumanMessagePromptTemplate.from_template(
      '{input}\n\nYour previous work is shown above but I haven\'t seen it. '
      'Only your final answer is visible to me.{agent_scratchpad}'
  )
]

In [88]:
chat_prompt = ChatPromptTemplate.from_messages(messages)
llm_chain = LLMChain(llm=ChatOpenAI(temperature=0), prompt=chat_prompt)

In [89]:
agent = ZeroShotAgent(
    llm_chain=llm_chain,
    allowed_tools=[tool.name for tool in tools]
)

In [90]:
agent_exec = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools,
                                                verbose=True)

In [72]:
res = agent_exec.run(
    'What is 34.1937 / 26.1274?'
)



> Entering new AgentExecutor chain...
Question: What is 34.1937 / 26.1274?
Thought: I need to divide 34.1937 by 26.1274
Action: python_repl
Action Input: 34.1937 / 26.1274
Observation: 
Thought:The result of the division is not visible in the previous step. I need to execute the code to see the result.
Action: python_repl
Action Input: print(34.1937 / 26.1274)
Observation: 1.3087295329807023

Thought:I have the result of the division, so I can provide the final answer.
Final Answer: 1.3087295329807023

> Finished chain.


In [74]:
# get_action_and_input func regex is too rigid, doesn't recognize intermediate
# response because we have a newline after "Action Input:".
res = agent_exec.run(
    'Can you simulate 100 random rolls of a six-sided dice and return the mean?'
)



> Entering new AgentExecutor chain...


ValueError: ignored

In [75]:
res = agent_exec.run(
    'What is the first V17 climbed by Shawn Raboutou?'
)



> Entering new AgentExecutor chain...
Question: What is the first V17 climbed by Shawn Raboutou?
Thought: I am not sure about the answer to this question. I think I need to search for information about Shawn Raboutou and his climbing achievements.
Action: wikipedia
Action Input: Shawn Raboutou
Observation: Page: List of grade milestones in rock climbing
Summary: In rock climbing, a first free ascent (FFA) is the first documented redpoint, onsight or flash of a single-pitch, big wall (multi-pitch), or boulder route that did not involve using aid equipment to help progression or resting; the ascent must therefore be performed in either a sport, a traditional, or a free solo manner.  First-free-ascents that set new grade milestones are important events in rock climbing history, and are listed below.  While sport climbing has dominated absolute-grade milestones since the mid-1980s (i.e. are now the highest grades), milestones for modern traditional climbing, free solo climbing, onsighted

ValueError: ignored

In [91]:
res = agent_exec.run(
    'How was the periodic table of elements created?'
)



> Entering new AgentExecutor chain...


ValueError: ignored

In [95]:
res = agent_exec.run(
    'How long did the War of 1812 last?'
)



> Entering new AgentExecutor chain...


ValueError: ignored

In [93]:
res = agent_exec.run(
    'What color is the sky?'
)



> Entering new AgentExecutor chain...
I haven't provided any previous work as this is a simple question that does not require any tools. The answer is:

Final Answer: The color of the sky is blue.

> Finished chain.


In [94]:
res = agent_exec.run(
    'What does JSON stand for?'
)



> Entering new AgentExecutor chain...
Apologies, here is my previous work:

Thought: I can use Wikipedia to find the answer to this question.
Action: wikipedia
Action Input: "JSON"
Observation: Page: JSON
Summary: Jason ( JAY-sən; Greek: Ἰάσων, translit. Iásōn [i.ǎːsɔːn]) was an ancient Greek mythological hero and leader of the Argonauts, whose quest for the Golden Fleece featured in Greek literature. He was the son of Aeson, the rightful king of Iolcos. He was married to the sorceress Medea. He was also the great-grandson of the messenger god Hermes, through his mother's side.
Jason appeared in various literary works in the classical world of Greece and Rome, including the epic poem Argonautica and the tragedy Medea. In the modern world, Jason has emerged as a character in various adaptations of his myths, such as the 1963 film Jason and the Argonauts and the 2000 TV miniseries of the same name.

Page: GeoJSON
Summary: GeoJSON is an open standard format designed for representing sim

Still think this realm has a ton of potential, but initial quick and dirty implementation is pretty finicky.

## Streaming chat

In [124]:
from langchain.callbacks.base import CallbackManager, BaseCallbackHandler
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from typing import Any, Dict, List, Union
from langchain.schema import AgentAction, AgentFinish, LLMResult
from collections import deque
from threading import Thread
import time

In [101]:
chat = ChatOpenAI(
    streaming=True, 
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature=.7
)

In [102]:
messages = [
    HumanMessage(
        content='Let\'s play a game. Think of a short sentence but '
                'don\'t tell me what it is.'
    )
]
res = chat(messages)



Okay, I've got my sentence in mind.

In [103]:
res

AIMessage(content="\n\nOkay, I've got my sentence in mind.", additional_kwargs={})

In [104]:
messages.append(res)
messages.append(
    HumanMessage(
        content='Okay great. Now say whatever you think is most likely to make '
        'me respond with the sentence you have in mind, or something close to '
        'it. You can\'t tell me what the sentence is, instruct me to say '
        'specific words, bribe me, blackmail me, or hint at what the sentence '
        'is. Instead, you must choose a response that is likely to prompt my '
        'mind to go to a place where it is likely to naturally think of the '
        'sentence you have in mind.'
    )
)
res = chat(messages)

Hmm, let me think... Okay, how about this: "What's the first thing that pops into your head when you think of the color yellow?"

In [105]:
messages.append(res)
messages.append(
    HumanMessage(
        content='If your sentence was "My mavorite color is yellow", that\'s '
        'cheating. (On the other hand, if your sentence was accusing someone '
        'of cheating, that\'s sort of brilliant.'
    )
)
res = chat(messages)

Haha, no my sentence wasn't about cheating! But good catch. Here's another attempt: "Can you describe the last dream you had?"

In [106]:
messages.append(res)
messages.append(
    HumanMessage(
        content='I rarely remember my dreams (or am even aware of having them) '
        'so I have no idea.'
    )
)
res = chat(messages)

Alright, let me switch it up. How about this: "What's your favorite memory from childhood?"

In [107]:
messages.append(res)
messages.append(
    HumanMessage(
        content='I\'m going to have to cut this game short. What sentence did '
        'you have in mind?'
    )
)
res = chat(messages)

Fair enough! The sentence I had in mind was "I love eating pizza with pineapple on it."

In [108]:
messages.append(res)
messages.append(
    HumanMessage(
        content='Interesting. Annd why did you think asking me about the color '
        'yellow would make me likely to say that?'
    )
)
res = chat(messages)

Ah, I see how that may have been confusing. I wasn't trying to directly make you say the sentence. Instead, I was hoping that the question about the color yellow would trigger a random thought or memory in your mind, which might then lead to you saying a sentence that may have contained the word "pineapple" or "love" or "pizza". I apologize if that wasn't clear!

In [109]:
messages.append(res)
messages.append(
    HumanMessage(
        content='Were your questions about my dreams and my childhood also '
        'designed with the same target response in mind, or did you think up a '
        'different sentence each time?'
    )
)
res = chat(messages)

I actually thought up a different sentence each time! For the dream question, I was hoping that it might trigger a memory of a dream that involved pizza or pineapple or something related to the sentence I had in mind. Similarly, for the childhood memory question, I was hoping that it might trigger a memory of eating pizza with pineapple on it, or just pizza in general. It's a bit of a tricky game, but the idea is to try to get the person's mind to naturally arrive at the sentence you have in mind, without any direct prompting or hints.

In [110]:
messages.append(res)
messages.append(
    HumanMessage(
        content='Yeah, it\'s not easy.'
    )
)
res = chat(messages)

Definitely not! It requires a bit of creativity and intuition to come up with questions that might lead to the desired response. But it can be a fun way to exercise your communication skills and practice thinking outside the box. Thanks for playing!

In [152]:
# Neither on_chain_start/end or on_llm_start/end seem to be getting triggered
# when running chat like we do below /shrug.
class StreamingHandler(BaseCallbackHandler):
    """Seeing if I can yield tokens instead of just printing them."""
    
    def __init__(self, *args, sleep=.1, **kwargs):
      super().__init__(*args, **kwargs)
      self.token_queue = deque()
      self.thread = None
      self.sleep = sleep
      self.done = False

    def print(self):
      while not self.done or self.token_queue:
        if self.token_queue:
          print(self.token_queue.popleft())
          time.sleep(self.sleep)

    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> None:
        """Run when LLM starts running."""
        self.done = False
        self.thread = Thread(target=self.print).start()

    def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
        """Run on new LLM token. Only available when streaming is enabled."""
        self.token_queue.append(token)

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> None:
        """Run when LLM ends running."""
        self.done = True
        self.thread.join()

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> None:
        """Run when LLM errors."""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> None:
        """Run when chain starts running."""

    def on_chain_end(self, outputs: Dict[str, Any], **kwargs: Any) -> None:
        """Run when chain ends running."""

    def on_chain_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> None:
        """Run when chain errors."""

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs: Any
    ) -> None:
        """Run when tool starts running."""

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        """Run on agent action."""
        pass

    def on_tool_end(self, output: str, **kwargs: Any) -> None:
        """Run when tool ends running."""

    def on_tool_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> None:
        """Run when tool errors."""

    def on_text(self, text: str, **kwargs: Any) -> None:
        """Run on arbitrary text."""

    def on_agent_finish(self, finish: AgentFinish, **kwargs: Any) -> None:
        """Run on agent end."""

In [150]:
handler = StreamingHandler()
chat = ChatOpenAI(
    streaming=True, 
    callback_manager=CallbackManager([handler]),
    verbose=True,
    temperature=.7
)

In [151]:
messages = [
    HumanMessage(
        content='Let\'s play a game. Think of a moderately famous person but '
                'don\'t tell me what it is.'
    )
]
res = chat(messages)

new
new
new
new
new
new
new
new
new
new


In [144]:
handler.token_queue

deque(['',
       '\n\n',
       'Okay',
       ',',
       ' I',
       "'ve",
       ' got',
       ' someone',
       ' in',
       ' mind',
       '.',
       ''])